# Introduction

This IPython notebook represents my written report and source code for COMPSCI 369 Assignment 2. To generate a PDF version of this report, simply type `rake` at the command line. The generated report can be found at `report.pdf`. Please note that you will need the following software installed.

* IPython 3
* \LaTeX
* pandoc
* Rake

In [ ]:
import itertools as it
import math
import random

BASES = ['A', 'C', 'G', 'T']
GAP = '-'

# Drawing from probability distributions

We can sample an integer uniformly at random from $\left\{0,1,\ldots,n-1\right\}$ by drawing $u$ uniformly at random from the interval $\left[0, 1\right)$ and transforming it by $$T\left(u\right) = \lfloor nu \rfloor.$$

In [ ]:
def random_int(i, n=1):
    if n > 1:
        return [math.floor(random.random() * i) for _ in range(n)]
    else:
        return math.floor(random.random() * i)

We can sample from a collection uniformly at random by drawing a random index and selecting the element associated with that index.

In [ ]:
def random_choice(X, n=1):
    if n > 1:
        return [X[i] for i in random_int(len(X), n)]
    else:
        return X[random_int(len(X))]

We can sample from the exponential distribution with rate $\lambda$ by drawing $u$ uniformly at random from the interval $\left[0, 1\right)$ and transforming it by $$F^{-1}\left(u\right) = -\frac{\ln{\left(1 - u\right)}}{\lambda},$$ where $F\left(x\right)$ is the cumulative distribution function of the exponential distribution.

In [ ]:
def random_exponential(lambd):
    return - math.log(1 - random.random()) / lambd

It is straightforward to simulate event times of a Poisson process with rate $\lambda$ by keeping a running sum of waiting times sampled from an exponential distribution with rate $\lambda$.

In [ ]:
def waiting_times(lambd):
    while True: yield random_exponential(lambd)

def poisson_process(lambd):
    return it.accumulate(waiting_times(lambd))

We can sample from the Poisson distribution with parameter $\lambda$ by simulating a Poisson process with rate $\lambda$ for 1 time unit and counting the number of events.

In [ ]:
def random_poisson(lambd):
    return sum(1 for _ in it.takewhile(lambda x: x < 1, poisson_process(lambd)))

# Question 1

We draw $L$ bases from the discrete uniform distribution on $\{A, C, G, T\}$ to get the ancestral sequence.

In [ ]:
def random_sequence(L):
    return ''.join(random_choice(BASES, L))

To simulate the evolution of a sequence $S$ with per-site mutation rate $\mu$, let $\lambda = L\frac{3}{4}\mu$ be the rate of observable  mutations (i.e., base $x$ to $y$ where $x \neq y$) across the entire sequence. We draw the number of mutations over time $t$ from a Poisson distribution with parameter $t \lambda$. For each mutation, we select the affected site uniformly at random and change it to one of the three other bases, selected uniformly at random.

In [ ]:
def evolve_sequence(S, mu, t):
    S = list(S) # Strings are immutable so use list of chars
    L = len(S)
    lambd = L * 3/4 * mu
    for _ in range(random_poisson(t * lambd)):
        i = random_int(L)
        S[i] = random_choice([b for b in BASES if b != S[i]])
    return ''.join(S) # Back to string

Finally, to simulate a pair of "sibling" sequences that have diverged from a common ancestor $t$ time units ago, first we draw an ancestral sequence $A$ and then simulate two independent evolutionary processes starting with $A$ and generating $B$ and $C$.

In [ ]:
def simulate_siblings(L, mu, t):
    A = random_sequence(L)
    B = evolve_sequence(A, mu, t)
    C = evolve_sequence(A, mu, t)
    return A, B, C

Here, we simulate a pair of sequences with length $L = 50$ and mutation rate $\mu = 0.01$ for $t = 10$ time units.

In [ ]:
L = 50
mu = 0.01
t = 10
A, B, C = simulate_siblings(L, mu, t)

In [ ]:
A

The sequence for the first child is

In [ ]:
B

and the sequence for the second child is

In [ ]:
C

The number of differences between the ancestor and child 1, the ancestor and child 2, and children 1 and 2 are, respectively,

In [ ]:
def count_differences(A, B):
    return sum(1 for x, y in zip(A, B) if x != y)
count_differences(A, B), count_differences(A, C), count_differences(B, C)

The mean, or expected value, of the Poisson distribution is equal to its parameter. Furthermore, the sum of independent Poisson variables is also Poisson distributed by the sum of their parameters. Therefore, the expected number of mutations for a single evolutionary process is $t\lambda = tL\frac{3}{4}\mu$ and the expected number of mutations between two sibling sequences is $2tL\frac{3}{4}\mu$.

Here, we simulate $n = 1000$ pairs of sequences with length $L = 1000$ and mutation rate $\mu = 0.01$ for $t = 25$ time units and count the number of sites at which they differ.

In [ ]:
n = 1000
L = 1000
mu = 0.01
t = 25
mean = 0
var = 0
for i in range(n):
    _, B, C = simulate_siblings(L, mu, t)
    d = count_differences(B, C)
    mean += d
    var += d * d
mean /= n
var /= n
var -= mean * mean

The mean number of differing sites is

In [ ]:
mean

and its variance is

In [ ]:
var

Although the number of mutations is Poisson distributed with parameter $2tL\frac{3}{4}\mu$, the number of differing sites is not. The number of mutations is not equivalent to the number of differing sites because more than one mutation can occur at a single site.

Here, we simulate a pair of sibling sequences $B$ and $C$ with length $L = 10000$ and mutation rate $\mu = 0.03$ for $t = 10$ time units. For this simulation, assuming that $p_{ab} = p_{ba}$, the empirical probabilities $p_{ab}$ are given by
$$p_{ab} = \frac{1}{2}\left(\frac{\left\vert{\left\{i \in \left\{1,\ldots,L\right\} \mid B_i = a \land C_i = b \right\}}\right\vert}{\left\vert{\left\{i \in \left\{1,\ldots,L\right\} \mid B_i = a\right\}}\right\vert} + \frac{\left\vert{\left\{i \in \left\{1,\ldots,L\right\} \mid B_i = b \land C_i = a \right\}}\right\vert}{\left\vert{\left\{i \in \left\{1,\ldots,L\right\} \mid B_i = b\right\}}\right\vert}\right)$$
and the theoretical by
$$p_{ab} = \begin{cases}\frac{1}{4} + \frac{3}{4}\exp{\left(-2t\mu\right)} &\text{if } a = b \\ \frac{1}{4} - \frac{1}{4}\exp{\left(-2t\mu\right)} &\text{if } a \neq b \end{cases}.$$

In [ ]:
L = 10000
mu = 0.03
t = 10
_, B, C = simulate_siblings(L, mu, t)
empirical_p = {}
theoretical_p = {}
for a,b in it.product(BASES, repeat=2):
    empirical_p[(a,b)] = (sum(1 for x,y in zip(B,C) if (x,y) == (a,b)) / B.count(a)
                          + sum(1 for x,y in zip(B,C) if (x,y) == (b,a)) / B.count(b)) / 2
    theoretical_p[(a,b)] = 1/4 + (3/4 if a == b else -1/4) * math.exp(-2 * t * mu)

The empirical $p_{ab}$ values are

In [ ]:
empirical_p

The theoretical $p_{ab}$ values are

In [ ]:
theoretical_p

# Question 2

Consider the described insertion-deletion (indel) model. Let $\lambda = \mu + 2\nu$ be the total rate of mutations, insertions and deletions at a single site. Then the total combined rate of mutations, insertions and deletions in a sequence of length $L$ is $L\lambda$. If an insertion of length $k$ occurs at some time $T$, the total combined rate of mutations, insertions and deletions is $L\lambda$ immediately before $T$ and is $\left(L+k\right)\lambda$ immediately after $T$. Therefore the total number of events on the tree is no longer Poisson distributed because the event rate is not constant through time.

The following method extends the previous method `simulate_siblings()` by simulating the simplififed indel process.

In [ ]:
def simulate_siblings_indel(L, mu, t):
    A, B, C = map(list, simulate_siblings(L, mu, t))
    for X,Y in it.permutations([B, C]):
        lambd = L * t * mu / 10
        h_I = random_poisson(lambd)
        h_D = random_poisson(lambd)
        for _ in range(h_I):
            i = random_int(len(X) + 1)
            # Gaps for ancestor
            A[i:i] = [GAP] * 3
            # Insertion
            X[i:i] = list(random_sequence(3))
            # Gaps for sibling
            Y[i:i] = [GAP] * 3
        for _ in range(h_D):
            # Create map of non-gap indices to gap indices
            m = [i for i, x in enumerate(X) if x != GAP] + [len(X)]
            # Choose i from non-gap sites and map to gap index
            i = random_choice(m)
            j = min(i+3, len(X))
            # Gaps representing deletion
            X[i:j] = [GAP] * (j - i)
    # Remove sites that are all gaps
    A, B, C = map(list, zip(*((a,b,c) for a,b,c in zip(A,B,C) if not a == b == c == GAP)))
    A, B, C = map(''.join, [A, B, C])
    return A, B, C

Here, we simulate a pair of sibling sequences $B$ and $C$ descended from a common ancestor with sequence length $L = 50$ under mutation rate $\mu = 0.01$ for $t = 20$ time units.

In [ ]:
L = 50
mu = 0.01
t = 20
_, B, C = simulate_siblings_indel(L, mu, t)
print(B)
print(C)

# Question 3

Some simple modifications to the global alignment algorithm make it an overlap alignment algorithm. Specifically, (1) the $F$ matrix is intially all zeros, (2) the backtrack starts from the best-scoring entry in the lower-right boundary of the $F$ matrix, and (3) the backtrack ends when anywhere in the upper-left boundary of the $F$ matrix. Otherwise, the global recurrence relation is still used for populating the $F$ matrix.

In [ ]:
def align_overlap(A, B, S, d):
    
    # Initialise F matrix with zeros
    F = [[0] * (len(B)+1) for _ in range(len(A)+1)]
    
    # Fill F matrix by global recurrence relation
    for i in range(len(A)):
        for j in range(len(B)):
            match = F[i][j] + S[A[i]][B[j]]
            delete = F[i][j+1] + d
            insert = F[i+1][j] + d
            F[i+1][j+1] = max(match, delete, insert)
    
    # Iterator over boundary indices
    boundary = it.chain(((i,len(B)-1) for i in range(len(A))),
                        ((len(A)-1,j) for j in range(len(B))))
    
    # Find index for boundary entry with greatest score
    i, j = max(boundary, key=lambda ij: F[ij[0]][ij[1]])
    
    # Append non-overlapping sequence or gaps
    alignment_A = A[i+1:] if i+1 < len(A) else '-' * (len(B) - j - 1)
    alignment_B = B[j+1:] if j+1 < len(B) else '-' * (len(A) - i - 1)
    
    # Backtrack to form alignment for overlapping region
    while i >= 0 and j >= 0:
        if i >= 0 and j >= 0 and F[i+1][j+1] == F[i][j] + S[A[i]][B[j]]:
            alignment_A = A[i] + alignment_A
            alignment_B = B[j] + alignment_B
            i -= 1
            j -= 1
        elif i >= 0 and F[i+1][j+1] == F[i][j+1] + d:
            alignment_A = A[i] + alignment_A
            alignment_B = GAP + alignment_B
            i -= 1
        else: # j >= 0 and F[i+1][j+1] == F[i+1][j] + d
            alignment_A = GAP + alignment_A
            alignment_B = B[j] + alignment_B
            j -= 1
    
    # Append non-overlapping sequence or gaps
    alignment_A = (A[:i] if i > 0 else '-' * j) + alignment_A
    alignment_B = (B[:j] if j > 0 else '-' * i) + alignment_B
    
    return alignment_A, alignment_B

Here, we align the first 35 bases of B to the last 35 bases of C (as previously simulated under the indel process) using the score matrix
$$S_{xy} = \begin{cases} 2 &\text{if } x = y \\ -2 &\text{if } x \neq y\end{cases}$$ and the gap penalty $d = -3$.

In [ ]:
S = {x: {y: 2 if x == y else -2 for y in BASES} for x in BASES}
d = -3
Bp = B.replace(GAP, '')[:35]
Cp = C.replace(GAP, '')[-35:]
alignment_B, alignment_C = align_overlap(Bp, Cp, S, d)
print(alignment_B)
print(alignment_C)

Consider the same alignment with various gap penalties $d \in \left\{-4,-3,-2,-1\right\}$.

In [ ]:
for d in range(-4, 0):
    alignment_B, alignment_C = align_overlap(Bp, Cp, S, d)
    print('d = {}'.format(d))
    print(alignment_B)
    print(alignment_C)
    print()

The gap penalty $d = -3$ tends to give good results across various simulated datasets because it is able to identify the overlapping region despite indels without overcompensating for mutations by inserting several superficial gaps.